In [4]:
import pandas as pd
import re
import subprocess
import numpy as np
import optuna
from optuna.samplers import TPESampler
import sqlite3
import time

In [5]:
# load kinetic data
df_kinetic = pd.read_csv('kinetic_data.csv')
df_mol = pd.read_csv('kinetic_data.csv').filter(like='mol%')
df_power = pd.read_csv('kinetic_data.csv').filter(like='Power')

In [6]:
class ParameterOptimizer:
    def __init__(self):
        
        # define the experimental data
        self.exp_list = ['H2', 'CH4', 'C2H6', 'C2H4', 'C2H2', 'C3H8', 'C3H6', 'C4H10', 'C5H12', 'C']
        self.exp_values = [df_mol.iloc[0].tolist(),
                           df_mol.iloc[1].tolist(),
                           df_mol.iloc[2].tolist()]

        # dictionary to save the current parameters
        self.ori_para = {}
        self.init_para()  

        # define manipulated variables
        self.lr = 0.01 # bounds learning rate
        self.max_error = 1000 # maximum error when simulation is failed
        self.n_trials = 100 # number of optimization trials

        # dynamic bounds for parameters
        self.bnds = {}
        self.init_bnds()

        # SQLite database setup (separate from Optuna's database)
        self.db_path = 'detailed_results.db'  # detailed results
        self.optuna_db_path = 'optuna_study.db'  # Optuna study
        self.setup_database()

        # 기존 최적화 결과가 있으면 bounds 업데이트
        self.load_best_bnds_from_db()
        
    
    def init_bnds(self):
        '''initialize parameter bounds based on learning rate'''
        low = np.zeros(len(self.ori_para)) - self.lr
        high = np.zeros(len(self.ori_para)) + self.lr
        self.bnds = np.vstack((low, high)).T
    
    def update_bnds(self, best_params):
        '''update parameter bounds based on best parameters found so far'''
        for i in range(len(self.bnds)):
            value = best_params[i]
            new_low = value * (1 - self.lr)
            new_high = value * (1 + self.lr)

            # update the bounds with the new values compared to the current bounds
            self.bnds[i] = [min(self.bnds[i][0], new_low), max(self.bnds[i][1], new_high)]

    def load_best_bnds_from_db(self):
        '''update parameter bounds based on best parameters in the database'''
        try:
            conn = sqlite3.connect(self.db_path, timeout=30.0)
            cursor = conn.cursor()

            # looking for the trial with the lowest error
            cursor.execute("SELECT * FROM optimization_results WHERE error_value = (SELECT MIN(error_value) FROM optimization_results)")
            best_row = cursor.fetchone()

            if best_row:
                # get the column names
                cursor.execute("PRAGMA table_info(optimization_results)")
                columns = [col[1] for col in cursor.fetchall()]

                # extract parameter values
                best_params = {}
                for i, col_name in enumerate(columns):
                    if col_name.startswith('p') and col_name in self.ori_para:
                        best_params[col_name] = best_row[i]
                
                if best_params:
                    print(f"found the best performance! error value: {best_row[columns.index('error_value')]}")
                    print("update the bounds based on the best parameters")
                    self.update_bnds(best_params)

            conn.close()

        except sqlite3.OperationalError:
            print("no previous optimization results found. start with initial bounds")
        except Exception as e:
            print(f'error loading best bounds from datbase: {e}')
        finally:
            try:
                conn.close()
            except:
                pass
    
    def setup_database(self):
        """Setup SQLite database for storing optimization results"""
        try:
            conn = sqlite3.connect(self.db_path, timeout=30.0)
            cursor = conn.cursor()
            
            # Create parameters columns dynamically
            param_columns = ', '.join([f'{param} REAL' for param in self.ori_para.keys()])
            
            cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS optimization_results (
                trial_number INTEGER PRIMARY KEY,
                {param_columns},
                error_value REAL,
                simulation_time REAL,
                status TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
            ''')
            
            conn.commit()
            conn.close()
        except Exception as e:
            print(f"데이터베이스 설정 오류: {e}")
        finally:
            try:
                conn.close()
            except:
                pass

    def save_to_database(self, trial_number, params, error_value, sim_time, status):
        """Save optimization results to SQLite database"""
        max_retries = 3
        retry_delay = 0.1
        
        for attempt in range(max_retries):
            try:
                conn = sqlite3.connect(self.db_path, timeout=30.0)
                cursor = conn.cursor()
                
                columns = ['trial_number'] + list(params.keys()) + ['error_value', 'simulation_time', 'status']
                values = [trial_number] + list(params.values()) + [error_value, sim_time, status]
                
                placeholders = ', '.join(['?' for _ in values])
                columns_str = ', '.join(columns)
                
                # INSERT OR REPLACE를 사용하여 중복 trial_number 문제 해결
                cursor.execute(f'''
                INSERT OR REPLACE INTO optimization_results ({columns_str})
                VALUES ({placeholders})
                ''', values)
                
                conn.commit()
                conn.close()
                return  # 성공하면 함수 종료
                
            except sqlite3.OperationalError as e:
                if "database is locked" in str(e) and attempt < max_retries - 1:
                    print(f"데이터베이스 잠금, {retry_delay}초 후 재시도... (시도 {attempt + 1}/{max_retries})")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # 지수적 백오프
                    continue
                else:
                    print(f"데이터베이스 저장 오류: {e}")
                    break
            except Exception as e:
                print(f"데이터베이스 저장 중 예외 발생: {e}")
                break
            finally:
                try:
                    conn.close()
                except:
                    pass
            
    def init_para(self):
        with open('kinet_ori.inp', 'r') as f:
            content = f.readlines()
       
        for line in content:
            if '$ double precision, parameter :: p' in line:
                # extract p parameters and values
                pattern = r'p(\d+)\s*=\s*([\d\.\+\-d]+)'
                match = re.search(pattern, line)
                if match:
                    param_name = 'p' + match.group(1)
                    value = float(match.group(2).replace('d','e'))
                    self.ori_para[param_name] = value
                    
    def modify_parameter(self, param_name: str, new_value: float, Voltage: float) -> None:
        # modify the parameter value in the kinet.inp file
        with open(f'kinet{Voltage}.inp', 'r') as f:
            content = f.read()

        # modify the parameter
        pattern = rf'(parameter :: {param_name} = )([\d\.\+\-d]+)'
        new_value_str = f'{new_value:.4e}'.replace('e', 'd')
        content = re.sub(pattern, f'parameter :: {param_name} = {new_value_str}', content)
        
        with open(f'kinet{Voltage}.inp', 'w') as f:
            f.write(content)

    def run_prep(self, Voltage):
        kinet_path = f'kinet{Voltage}.inp'
        process = subprocess.Popen(
            './preprocessor.exe',
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=False)
        
        process.stdin.write(f'{kinet_path}\n'.encode())
        process.stdin.flush()

        process.stdin.write('.\n'.encode())
        process.stdin.flush()

        output, error = process.communicate()
        output_str = output.decode('utf-8', errors='ignore') if output else ''
        error_str = error.decode('utf-8', errors='ignore') if error else ''

        return output_str, error_str
    
    def compile_zdp(self, Voltage):
        exe_path = f'run_{Voltage}kV.exe'
        compile_command= [
            'gfortran', '-o', exe_path, 'dvode_f90_m.F90', 'zdplaskin_m.F90', f'run_{Voltage}kV.F90', 'bolsig_x86_64_g.dll'
        ]
        result = subprocess.run(compile_command, capture_output=True, text=True)

        if result.returncode != 0:
            raise Exception(f"{exe_path} 컴파일 실패")

    def run_zdplaskin(self,Voltage):
        exe_path = f'run_{Voltage}kV.exe'
        try:
            process = subprocess.Popen(
                exe_path,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                stdin=subprocess.PIPE,
                universal_newlines=True,
                bufsize=1,
            )

            while True:
                output = process.stdout.readline()

                if not output:
                    break
                print(f'\r{output.strip()}                                                                              ',end='',flush=True)

                if "PRESS ENTER TO EXIT" in output:
                    process.kill()
                    break

                if "WARNING: BOLSIG+ convergence failed" in output:
                    process.stdin.write('\n')
                    process.stdin.flush()

        except:
            pass
        return process
    
    def err_cal(self, Voltage):
        #read the result file
        species = []
        with open('qt_species_list.txt', 'r') as f:
            for line in f:
                comp = line[2:]
                species.append(comp.strip())
        
        df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
        
        # calculate the concentration
        H2 = (df_sp['H2'])
        CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
        C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'] + df_sp['C2H2(V13)'])
        C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])
        C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])
        C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
        C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
        C4H10 = (df_sp['C4H9H'])
        C5H12 = (df_sp['C5H12'])
        C = (df_sp['C'])
        CH2 = (df_sp['CH2'])
        CH3 = (df_sp['CH3'])
        C2H3 = (df_sp['C2H3'])
        C2H5 = (df_sp['C2H5'])
        C3H7 = (df_sp['C3H7'])
        H = (df_sp['H'])
        CH3_plus = (df_sp['CH3^+'])
        CH4_plus = df_sp['CH4^+']
        CH5_plus = df_sp['CH5^+']
        C2H2_plus = df_sp['C2H2^+']
        C2H4_plus = df_sp['C2H4^+']
        C2H5_plus = df_sp['C2H5^+']
        C2H6_plus = df_sp['C2H6^+']
        C3H6_plus = df_sp['C3H6^+']
        C3H8_plus = df_sp['C3H8^+']

        t = abs(df_sp['Time [s]']-16.9646).argmin()

        sim_H2 = float(format(H2.iloc[t], '.6f'))
        sim_CH4 = float(format(CH4.iloc[t], '.6f'))
        sim_C2H2 = float(format(C2H2.iloc[t], '.6f'))
        sim_C2H4 = float(format(C2H4.iloc[t], '.6f'))
        sim_C2H6 = float(format(C2H6.iloc[t], '.6f'))
        sim_C3H6 = float(format(C3H6.iloc[t], '.6f'))
        sim_C3H8 = float(format(C3H8.iloc[t], '.6f'))
        sim_C4H10 = float(format(C4H10.iloc[t], '.6f'))
        sim_C5H12 = float(format(C5H12.iloc[t], '.6f'))
        sim_C = float(format(C.iloc[t], '.6f'))
        sim_CH2 = float(format(CH2.iloc[t], '.6f'))
        sim_CH3 = float(format(CH3.iloc[t], '.6f'))
        sim_C2H3 = float(format(C2H3.iloc[t], '.6f'))
        sim_C2H5 = float(format(C2H5.iloc[t], '.6f'))
        sim_C3H7 = float(format(C3H7.iloc[t], '.6f'))
        sim_H = float(format(H.iloc[t], '.6f'))
        sim_CH3_plus = float(format(CH3_plus.iloc[t], '.6f'))
        sim_CH4_plus = float(format(CH4_plus.iloc[t], '.6f'))
        sim_CH5_plus = float(format(CH5_plus.iloc[t], '.6f'))
        sim_C2H2_plus = float(format(C2H2_plus.iloc[t], '.6f'))
        sim_C2H4_plus = float(format(C2H4_plus.iloc[t], '.6f'))
        sim_C2H5_plus = float(format(C2H5_plus.iloc[t], '.6f'))
        sim_C2H6_plus = float(format(C2H6_plus.iloc[t], '.6f'))
        sim_C3H6_plus = float(format(C3H6_plus.iloc[t], '.6f'))
        sim_C3H8_plus = float(format(C3H8_plus.iloc[t], '.6f'))

        out_H2 = sim_H2 - sim_CH2 - 0.5*sim_CH3 - 0.5*sim_CH3_plus + 0.5*sim_CH5_plus - 0.5*sim_C2H3 + 0.5*sim_C2H5 + 0.5*sim_C2H5_plus + 0.5*sim_C3H7 + 0.5*sim_H
        out_CH4 = sim_CH4 + sim_CH2 + sim_CH3 + sim_CH3_plus + sim_CH4_plus + sim_CH5_plus
        out_C2H6 = sim_C2H6 + sim_C2H6_plus
        out_C2H4 = sim_C2H4 + sim_C2H3 + sim_C2H5 + sim_C2H4_plus + sim_C2H5_plus
        out_C2H2 = sim_C2H2 + sim_C2H2_plus
        out_C3H8 = sim_C3H8 + sim_C3H8_plus
        out_C3H6 = sim_C3H6 + sim_C3H7 + sim_C3H6_plus
        out_C4H10 = sim_C4H10
        out_C5H12 = sim_C5H12
        out_C = sim_C
        out_total = out_H2 + out_CH4 + out_C2H6 + out_C2H4 + out_C2H2 + out_C3H8 + out_C3H6 + out_C4H10 + out_C5H12 + out_C
        newsim = [out_H2/out_total*100, out_CH4/out_total*100, out_C2H6/out_total*100, out_C2H4/out_total*100, out_C2H2/out_total*100, out_C3H8/out_total*100, out_C3H6/out_total*100, out_C4H10/out_total*100, out_C5H12/out_total*100, out_C/out_total*100]
        
        w_factor = [0, 1, 10, 100, 10, 10, 10, 10, 1, 0]
        err = 0

        if Voltage == 10:
            id_exp = 2
        elif Voltage == 12.5:
            id_exp = 1
        elif Voltage == 15:
            id_exp = 0

        for i in range(len(self.exp_values[id_exp])):
            err += w_factor[i] * ((self.exp_values[id_exp][i] - newsim[i]))**2

        return err, df_sp['Time [s]'].iloc[-1]
    
    def objective(self, trial):
        '''optuna objective function'''
        print(f'\n{'='*50}')
        print(f'Trial {trial.number} started')
        print(f'{'='*50}')

        # suggest parameters within current bounds
        alpha = {}
        for i, param_name in enumerate(self.ori_para.keys()):
            alpha[param_name] = trial.suggest_float(param_name, self.bnds[i][0], self.bnds[i][1])

        PD0 = 4.317
        Volt = [10, 12.5]
        terr = 0
        runtime = []

        try:
            for v in Volt:
                if v == 10:
                    PD = 2.64
                elif v == 12.5:
                    PD = 3.273
                
                para = self.ori_para.copy()
                for i, para_name in enumerate(para.keys()):
                    para[para_name] = np.exp(alpha[param_name]*(PD0-PD))
                
                print(f'set the parameters: {v} kV case')
                # modify parameters in the input file
                for para_name, value in para.items():
                    self.modify_parameter(para_name, value, v)
                
                print(f'Run {v}kV')
                self.run_prep(v)
                self.compile_zdp(v)
                self.run_zdplaskin(v)
                err,time = self.err_cal(v)
                terr += err
                runtime.append(time)
                print('')

            # check if simulation completed successfully
            if min(runtime) < 17.0:
                terr = self.max_error
                status = 'simulation_incomplete'
                print(f'simulation is incomplete (min time: {min(runtime):.2f}s)')
            else:
                status = 'success'
                print(f'simulation is completed (min time: {min(runtime):.2f}s)')

            print(f'Trial {trial.number} error: {terr:.6f}')

            # save results to database
            self.save_to_database(trial.number, alpha, terr, min(runtime), status)

        except Exception as e:
            terr = self.max_error
            sim_time = 0
            status = f"error: {str(e)}"
            print(f"Error in running Trial {trial.number}: {e}")
            # try to save the results even if error occurs
            try:
                self.save_to_database(trial.number, alpha, terr, sim_time, status)
            except Exception as db_error:
                print(f"Failed to save results to database: {db_error}")

        print(f"Trial {trial.number} completed\n")
        return terr
    
    def update_bounds_callback(self, study, trial):
        """Callback to update bounds when a new best trial is found"""
        if study.best_trial.number == trial.number:
            # Update bounds based on the new best parameters
            self.update_bnds(study.best_params)

    def optimize(self):
        """Run optimization using Optuna"""
        # Create study with TPE sampler and SQLite storage
        study = optuna.create_study(
            direction='minimize',
            sampler=TPESampler(seed=42, n_startup_trials=10),
            storage=f"sqlite:///{self.optuna_db_path}",
            study_name='parameter_optimization',
            load_if_exists=True
        )

        # check if there is any existing optimization
        n_existing_trials = len(study.trials)
        print(f'existing Optuna trials: {n_existing_trials} trials')

        # if there is any existing optimization, update bounds based on the best parameters
        if n_existing_trials > 0:
            try:
                best_params = study.best_params
                print(f'found the best performance! error value: {study.best_value}')
                print('update the bounds based on the best parameters')
                self.update_bnds(best_params)
            except:
                print('failed to load best parameters from database')
        
        # Optimize with callback for dynamic bounds update
        study.optimize(
            self.objective, 
            n_trials=self.n_trials,
            callbacks=[self.update_bounds_callback]
        )
        
        return study
    
if __name__ == "__main__":
    optimizer = ParameterOptimizer()

    print("Starting parameter optimization with Optuna...")
    print(f"Detailed results database: {optimizer.db_path}")
    print(f"Optuna database: {optimizer.optuna_db_path}")
    print(f"Number of parameters: {len(optimizer.ori_para)}")
    print(f"Maximum trials: {optimizer.n_trials}")
    print(f"Learning rate: {optimizer.lr}")
    
    study = optimizer.optimize()
    
    print("\nOptimization completed!")
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best error: {study.best_value}")
    print("Best parameters saved in database.")        

[I 2025-06-27 17:05:58,903] Using an existing study with name 'parameter_optimization' instead of creating a new one.


Starting parameter optimization with Optuna...
Detailed results database: detailed_results.db
Optuna database: optuna_study.db
Number of parameters: 32
Maximum trials: 100
Learning rate: 0.01
existing Optuna trials: 1 trials
found the best performance! error value: 115.80463161165125
update the bounds based on the best parameters
failed to load best parameters from database

Trial 1 started
set the parameters: 10 kV case
Run 10kV
PRESS ENTER TO EXIT ...At line 89 of file run_10kV.F90 (unit = 5, file = 'fort.5')                                                                                     
set the parameters: 12.5 kV case
Run 12.5kV
PRESS ENTER TO EXIT ...At line 89 of file run_12.5kV.F90 (unit = 5, file = 'fort.5')                                                                                   
simulation is completed (min time: 17.10s)
Trial 1 error: 115.804632


[I 2025-06-27 17:06:26,696] Trial 1 finished with value: 115.80463161165125 and parameters: {'p0': -0.0025091976230527504, 'p1': 0.009014286128198325, 'p2': 0.0046398788362281024, 'p3': 0.0019731696839407325, 'p4': -0.006879627191151269, 'p5': -0.006880109593275947, 'p6': -0.00883832775663601, 'p7': 0.007323522915498705, 'p8': 0.002022300234864176, 'p9': 0.00416145155592091, 'p10': -0.009588310114083952, 'p11': 0.009398197043239886, 'p12': 0.006648852816008436, 'p13': -0.005753217786434477, 'p14': -0.006363500655857988, 'p15': -0.006331909802931324, 'p16': -0.003915155140809245, 'p17': 0.000495128632644757, 'p18': -0.0013610996271576845, 'p19': -0.004175417196039162, 'p20': 0.0022370578944475895, 'p21': -0.007210122786959164, 'p22': -0.004157107029295637, 'p23': -0.0026727631341261658, 'p24': -0.0008786003156592819, 'p25': 0.005703519227860271, 'p26': -0.0060065243568328056, 'p27': 0.00028468876827223225, 'p28': 0.0018482913772408498, 'p29': -0.009070991745600046, 'p30': 0.002150897038

Trial 1 completed


Trial 2 started
set the parameters: 10 kV case
Run 10kV
PRESS ENTER TO EXIT ...At line 89 of file run_10kV.F90 (unit = 5, file = 'fort.5')                                                                                     
set the parameters: 12.5 kV case
Run 12.5kV
PRESS ENTER TO EXIT ...At line 89 of file run_12.5kV.F90 (unit = 5, file = 'fort.5')                                                                                   
simulation is completed (min time: 17.10s)
Trial 2 error: 117.417773


[I 2025-06-27 17:06:54,204] Trial 2 finished with value: 117.41777272674678 and parameters: {'p0': -0.00869896814029441, 'p1': 0.008977710745066666, 'p2': 0.009312640661491187, 'p3': 0.006167946962329222, 'p4': -0.003907724616532587, 'p5': -0.008046557719872323, 'p6': 0.0036846605302431387, 'p7': -0.0011969501252079746, 'p8': -0.007559235303104424, 'p9': -9.646179777459546e-05, 'p10': -0.009312229577695633, 'p11': 0.008186408041575643, 'p12': -0.0048244003679996615, 'p13': 0.003250445687079639, 'p14': -0.003765778478211781, 'p15': 0.00040136042355621605, 'p16': 0.000934205586865593, 'p17': -0.00630291088948946, 'p18': 0.009391692555291173, 'p19': 0.005502656467222292, 'p20': 0.008789978831283784, 'p21': 0.007896547008552976, 'p22': 0.0019579995762217025, 'p23': 0.008437484700462338, 'p24': -0.00823014995896161, 'p25': -0.0060803427516170966, 'p26': -0.009095454221789239, 'p27': -0.003493393384734713, 'p28': -0.0022264542062103597, 'p29': -0.004573019364522082, 'p30': 0.0065747501830385

Trial 2 completed


Trial 3 started
set the parameters: 10 kV case
Run 10kV
PRESS ENTER TO EXIT ...At line 89 of file run_10kV.F90 (unit = 5, file = 'fort.5')                                                                                     
set the parameters: 12.5 kV case
Run 12.5kV
PRESS ENTER TO EXIT ...At line 89 of file run_12.5kV.F90 (unit = 5, file = 'fort.5')                                                                                   
simulation is completed (min time: 17.10s)
Trial 3 error: 118.617412
Trial 3 completed



[I 2025-06-27 17:07:22,560] Trial 3 finished with value: 118.61741233913591 and parameters: {'p0': -0.004381309806252385, 'p1': 0.0008539216631649697, 'p2': -0.007181515500504747, 'p3': 0.006043939615080792, 'p4': -0.008508987126404584, 'p5': 0.009737738732010346, 'p6': 0.005444895385933148, 'p7': -0.006025686369316552, 'p8': -0.009889557657527952, 'p9': 0.006309228569096685, 'p10': 0.004137146876952342, 'p11': 0.004580143360819746, 'p12': 0.005425406933718915, 'p13': -0.008519106965318194, 'p14': -0.002830685429114548, 'p15': -0.007682618809497405, 'p16': 0.007262068517511872, 'p17': 0.002465962536551158, 'p18': -0.0033820395029470164, 'p19': -0.008728832994279527, 'p20': -0.0037803535656867563, 'p21': -0.0034963335594650593, 'p22': 0.004592123566761282, 'p23': 0.002751149427104263, 'p24': 0.007744254851526531, 'p25': -0.0005557014967610147, 'p26': -0.007608115081233966, 'p27': 0.004264895744459899, 'p28': 0.005215700972337948, 'p29': 0.0012255439513899243, 'p30': 0.005419343599091219


Trial 4 started
set the parameters: 10 kV case
Run 10kV


[W 2025-06-27 17:07:27,053] Trial 4 failed with parameters: {'p0': 0.00045465658763988066, 'p1': -0.001449179632829008, 'p2': -0.009491617465118096, 'p3': -0.007842171460133912, 'p4': -0.009371416286265316, 'p5': 0.0027282082252756083, 'p6': -0.003712880378473467, 'p7': 0.00017141382329405493, 'p8': 0.008151329478521862, 'p9': -0.005014155417022501, 'p10': -0.0017923415392874058, 'p11': 0.005111022770860974, 'p12': -0.005424036690167551, 'p13': -0.00846040180342414, 'p14': -0.00420497094172464, 'p15': -0.006775574254919912, 'p16': 0.00859395304685146, 'p17': 0.006162407591288338, 'p18': 0.0026680751302084697, 'p19': 0.0074292118037543545, 'p20': 0.006073441537982288, 'p21': -0.006268598822279283, 'p22': 0.007851179969799555, 'p23': 0.0007868448383130146, 'p24': 0.006148803103281251, 'p25': 0.007921825998469864, 'p26': -0.0036399305005627228, 'p27': -0.007798961509446465, 'p28': -0.005441296749161166, 'p29': -0.0014578442274748738, 'p30': 0.006360295318449864, 'p31': 0.00721461166512686

KeyboardInterrupt: 